# Chapter 12 Analysis of Single Factor Experiments

In [1]:
import polars as pl
from polars import col, lit
from scipy import stats
import numpy as np
import altair as alt

RNG = np.random.default_rng()
DATA = {}  # input data
ANS = {}   # calculation results

## 12.1 Completely Randomized Design

To calculate the confidence interval of the group mean using a _pooled_ $\hat s$:
$$
\text{CI} = \hat\mu_\text{group} \pm t_{N-a,\ \alpha/2} \frac{\hat{s}_\text{pool}}{\sqrt{n_\text{group}}}\ .
$$

If there are multiple datasets, the CIs are calculated independently for each dataset.

In [2]:
def CI(data: pl.DataFrame, α: float=0.05) -> pl.DataFrame:
    """ 
    returns the studentized CIs for each group in the data using a pooled s.

    input columns:
    - dataset: name of dataset. can be empty if only one dataset.
    - group: name of treatment groups
    - n: group size
    - μ: group mean
    - s: group standard deviation

    output columns:
    - dataset
    - group
    - CI: a pair of floats
    """
    dof = col('n').sum() - pl.len()
    t_crit = dof.map_elements(lambda v: stats.t.ppf(1-α/2, v))
    s_pool = ((col('s')**2 * (col('n') - 1)).sum() / dof).sqrt()
    se = s_pool / col('n').sqrt()
    
    return (
        data.group_by('dataset')
        .agg(
            'group',
            pl.concat_list(col('μ') - t_crit*se, col('μ') + t_crit*se)
            .alias('CI'))
        .explode('group', 'CI'))

To generate the ANOVA table for each dataset. Follows the textbook.

In [3]:
def ANOVA(data: pl.DataFrame) -> pl.DataFrame:
    """ 
    returns the 1-way ANOVA table as a DataFrame.

    assumed conditions in the input data:
    - normality assumption
    - constant variance

    input columns:
    - dataset: string id of dataset
    - n: group size
    - μ: group mean
    - s: group standard deviation

    output columns:
    - dataset
    - standard ANOVA table columns: 
        - source: treatment, error, total
        - SS: sum of squares
        - dof: degree of freedom
        - MS: mean squres
        - F: F statistic
    """
    μ_total = (col('μ') * col('n')).sum() / col('n').sum()
    
    # factor A sum of squares
    ssa = ((col('μ') - μ_total)**2 * col('n')).sum()
    dof_ssa = pl.len() - 1
    
    # error sum of squares
    sse = (col('s')**2 * (col('n') - 1)).sum()
    dof_sse = col('n').sum() - pl.len()
    
    msa = ssa / dof_ssa # factor A mean square
    mse = sse / dof_sse # error mean square

    return (
        data
        .group_by('dataset')
        .agg(ssa=ssa, dof_ssa=dof_ssa, sse=sse, dof_sse=dof_sse, msa=msa, mse=mse)
        .select(
            'dataset', 
            source=pl.concat_list(lit('treatment'), lit('error'), lit('total')),
            SS=pl.concat_list('ssa', 'sse', col('ssa') + col('sse')),
            dof=pl.concat_list('dof_ssa', 'dof_sse', col('dof_ssa') + col('dof_sse')),
            MS=pl.concat_list('msa', 'mse', None),
            F=pl.concat_list(col('msa') / col('mse'), None, None))
        .explode('source', 'SS', 'dof', 'MS', 'F'))

## 12.1 Solutions to Exercises

### 1.

In [4]:
DATA['12.1'] = pl.read_json('Ex12-1.json').explode(pl.all())
DATA['12.1'].head()

n,SugarAvg,SugarSD,FiberAvg,FiberSD,Location
f64,f64,f64,f64,f64,str
20.0,4.8,2.138,1.68,1.166,"""Shelf1"""
20.0,9.85,1.985,0.95,1.162,"""Shelf2"""
20.0,6.1,1.865,2.17,1.277,"""Shelf3"""


Transform to a long format that is compatible with the function `CI`. The shelves correspond to the treatment groups and there are 2 datasets: sugar and fiber.

In [5]:
DATA['12.1 - long'] = (
    DATA['12.1'].select(
        group='Location',
        n=col('n').cast(int),
        container=pl.concat_list(
            pl.struct(dataset=lit('sugar'), μ='SugarAvg', s='SugarSD'),
            pl.struct(dataset=lit('fiber'), μ='FiberAvg', s='FiberSD')))
    .explode('container')
    .unnest('container')
    .sort('dataset', 'group'))

DATA['12.1 - long']

group,n,dataset,μ,s
str,i64,str,f64,f64
"""Shelf1""",20,"""fiber""",1.68,1.166
"""Shelf2""",20,"""fiber""",0.95,1.162
"""Shelf3""",20,"""fiber""",2.17,1.277
"""Shelf1""",20,"""sugar""",4.8,2.138
"""Shelf2""",20,"""sugar""",9.85,1.985
"""Shelf3""",20,"""sugar""",6.1,1.865


#### (a)

In [6]:
ANS['12.1.a'] = CI(DATA['12.1 - long'])
ANS['12.1.a']

dataset,group,CI
str,str,list[f64]
"""fiber""","""Shelf1""","[1.141407, 2.218593]"
"""fiber""","""Shelf2""","[0.411407, 1.488593]"
"""fiber""","""Shelf3""","[1.631407, 2.708593]"
"""sugar""","""Shelf1""","[3.904862, 5.695138]"
"""sugar""","""Shelf2""","[8.954862, 10.745138]"
"""sugar""","""Shelf3""","[5.204862, 6.995138]"


Let's visualize the CIs.

In [7]:
(
    alt.Chart(
        ANS['12.1.a'].select('dataset', 'group',
            low=col('CI').list.first(),
            high=col('CI').list.last()))
    .mark_line(strokeWidth=2)
    .encode(
        alt.X('low').title('CI'),
        alt.X2('high'),
        alt.Y('group').title(None),
        alt.Row('dataset').title(None))
    .resolve_scale(x='independent')
)

alt.Chart(...)

Reading from the chart:
- Sugar: The CIs of Shelf 1 and Shelf 3 overlap, so no obvious differences. But Shelf 2 stands out from the rest.
- Fiber: By the same token, Shelf 2 and Shelf 3 are different because their CIs do not overlap.

#### (b)

In [8]:
ANS['12.1.b'] = ANOVA(DATA['12.1 - long'])
ANS['12.1.b']

dataset,source,SS,dof,MS,F
str,str,f64,i64,f64,f64
"""fiber""","""treatment""",15.076,2,7.538,5.209964
"""fiber""","""error""",82.470051,57,1.446843,null
"""fiber""","""total""",97.546051,59,null,null
"""sugar""","""treatment""",275.033333,2,137.516667,34.409292
"""sugar""","""error""",227.800386,57,3.996498,null
"""sugar""","""total""",502.833719,59,null,null


The 95% critical value $f(2, 57)$ is

In [9]:
stats.f(2, 57).ppf(0.95)

3.1588427192606465

Therefore both the sugar and fiber content F ratios (34.4 and 5.2 respectively) are greater than the critical value, meaning there are significant differences among the shelves.

#### (c)

Shelf 2 contains cereals that are high in sugar and low in fiber (refer to the line chart in (b)), in other words, "taste good". The grocery store's strategy is to place those where grade schoolers can easily see.

### 2.

In [10]:
DATA['12.2'] = pl.read_json('Ex12-2.json').explode(pl.all())
DATA['12.2'].head()

mg,taprate
f64,f64
0.0,242.0
0.0,245.0
0.0,244.0
0.0,248.0
0.0,247.0


In [11]:
DATA['12.2 - clean'] = (
    DATA['12.2']
    .select(
        pl.format('{} mg', col('mg').cast(int)),
        col('taprate').cast(int)))

DATA['12.2 - clean'].head()

mg,taprate
str,i64
"""0 mg""",242
"""0 mg""",245
"""0 mg""",244
"""0 mg""",248
"""0 mg""",247


#### (a)

In [12]:
(
    alt.Chart(DATA['12.2 - clean'])
    .mark_boxplot()
    .encode(
        alt.Y('mg'),
        alt.X('taprate').scale(zero=False))
)

alt.Chart(...)

The chart seems to indicate different effects among the doses.

#### (b)

In [13]:
DATA['12.2 - aggregated'] = (
    DATA['12.2']
    .group_by(col('mg').alias('group'))
    .agg(
        n=pl.len(),
        μ=col('taprate').mean(),
        s=col('taprate').std())
    .with_columns(dataset=pl.lit(''))) # only one dataset so don't bother naming

ANS['12.2.b'] = ANOVA(DATA['12.2 - aggregated'])
ANS['12.2.b']

dataset,source,SS,dof,MS,F
str,str,f64,u32,f64,f64
"""""","""treatment""",61.4,2,30.7,6.181208
"""""","""error""",134.1,27,4.966667,null
"""""","""total""",195.5,29,null,null


On the other hand:

In [14]:
stats.f(2, 27).ppf(0.9)

2.5106086665585408

And 2.51 < 6.18. So yes, there are significant differences.

Corroboration:

In [15]:
stats.f_oneway(*(g['taprate'] for g in DATA['12.2 - clean'].partition_by('mg')))

F_onewayResult(statistic=6.181208053691275, pvalue=0.006163213574095554)

#### (c)

In [16]:
chart_base = (
    alt.Chart(
        DATA['12.2 - clean']
        .with_columns(
            residual=col('taprate') - col('taprate').mean().over('mg'))
        .with_columns(
            normal_score=(col('residual').rank() / (pl.len() + 1)).map_batches(
                lambda x: stats.norm.ppf(x))))
    .mark_circle())

(
    chart_base.encode(
        alt.X('mg').axis(labelAngle=0).title(None),
        alt.Y('residual'))
    .properties(width=200)
    | chart_base.encode(
        x='residual', 
        y='normal_score')
)

alt.HConcatChart(...)

From the charts: the variances are consistent across different doses. The residuals are fairly normally distributed.

### 3.

In [17]:
DATA['12.3'] = pl.read_json('Ex12-3.json').explode(pl.all())
DATA['12.3'].head()

AvgEgg,Group
f64,str
35.4,"""Control"""
27.4,"""Control"""
19.3,"""Control"""
41.8,"""Control"""
20.3,"""Control"""


#### (a)

In [18]:
(
    alt.Chart(DATA['12.3'])
    .mark_boxplot()
    .encode(y='Group', x='AvgEgg')
)

alt.Chart(...)

The control group is a bit higher in terms of average eggs laid per day.

#### (b)

In [19]:
DATA['12.3 - aggregated'] = (
    DATA['12.3']
    .group_by('Group')
    .agg(
        n=pl.len(),
        μ=col('AvgEgg').mean(),
        s=col('AvgEgg').std())
    .with_columns(dataset=pl.lit('')))

ANS['12.3.b'] = ANOVA(DATA['12.3 - aggregated'])
ANS['12.3.b']

dataset,source,SS,dof,MS,F
str,str,f64,u32,f64,f64
"""""","""treatment""",1362.211467,2,681.105733,8.665739
"""""","""error""",5659.0224,72,78.597533,null
"""""","""total""",7021.233867,74,null,null


In [20]:
stats.f(2, 72).ppf(0.95)

3.1239074485457783

8.67 > 3.12, so the difference is significant.

#### (c)

In [21]:
chart_base = (
    alt.Chart(
        DATA['12.3']
        .with_columns(
            residual=col('AvgEgg') - col('AvgEgg').mean().over('Group'))
        .with_columns(
            normal_score=(col('residual').rank() / (pl.len() + 1)).map_batches(
                lambda x: stats.norm.ppf(x))))
    .mark_circle())

(
    chart_base.encode(
        alt.X('Group').axis(labelAngle=0).title(None),
        alt.Y('residual'))
    .properties(width=200)
    | chart_base.encode(
        x='residual', 
        y='normal_score')
)

alt.HConcatChart(...)

From the charts: the variances are consistent across different groups. The residuals are fairly normally distributed.

### 4.

In [22]:
DATA['12.4'] = pl.read_json('Ex12-4.json').explode(pl.all())
DATA['12.4'].head()

salinity,site
f64,str
37.54,"""I"""
37.01,"""I"""
36.71,"""I"""
37.03,"""I"""
37.32,"""I"""


#### (a)

In [23]:
(
    alt.Chart(DATA['12.4'])
    .mark_boxplot()
    .encode(
        alt.Y('site'),
        alt.X('salinity').scale(zero=False))
)

alt.Chart(...)

Obviously, site II > site III > site I in terms of salinity.

#### (b)

In [24]:
DATA['12.4 - aggregated'] = (
    DATA['12.4']
    .group_by(col('site').alias('group'))
    .agg(
        n=pl.len(),
        μ=col('salinity').mean(),
        s=col('salinity').std())
    .with_columns(dataset=pl.lit('')))

ANS['12.4.b'] = ANOVA(DATA['12.4 - aggregated'])
ANS['12.4.b']

dataset,source,SS,dof,MS,F
str,str,f64,u32,f64,f64
"""""","""treatment""",38.800883,2,19.400441,66.021046
"""""","""error""",7.934014,27,0.293852,null
"""""","""total""",46.734897,29,null,null


In [25]:
stats.f(2, 27).ppf(0.99)

5.488117768420701

So the difference in salinity is indeed significant among the 3 sites.

#### (c)

In [26]:
chart_base = (
    alt.Chart(
        DATA['12.4']
        .with_columns(
            residual=col('salinity') - col('salinity').mean().over('site'))
        .with_columns(
            normal_score=(col('residual').rank() / (pl.len() + 1)).map_batches(
                lambda x: stats.norm.ppf(x))))
    .mark_circle())

(
    chart_base.encode(
        alt.X('site').axis(labelAngle=0),
        alt.Y('residual'))
    .properties(width=200)
    | chart_base.encode(
        alt.X('residual'), 
        alt.Y('normal_score'))
)

alt.HConcatChart(...)

From the charts: the variances are roughly consistent across different sites. The residuals are mostly normally distributed, with a few points maybe too large.

### 5.

In [27]:
DATA['12.5'] = pl.read_json('Ex12-5.json').explode(pl.all())
DATA['12.5'].head()

hemoglbn,Group
f64,str
7.2,"""HBSS"""
7.7,"""HBSS"""
8.0,"""HBSS"""
8.1,"""HBSS"""
8.3,"""HBSS"""


#### (a)

In [28]:
(
    alt.Chart(DATA['12.5'])
    .mark_boxplot()
    .encode(
        alt.Y('Group'),
        alt.X('hemoglbn').scale(zero=False))
)

alt.Chart(...)

The groups seem to differ in hemoglobin levels: HBSC > HBS > HBSS.

#### (b)

In [29]:
DATA['12.5 - aggregated'] = (
    DATA['12.5']
    .group_by(col('Group').alias('group'))
    .agg(
        n=pl.len(),
        μ=col('hemoglbn').mean(),
        s=col('hemoglbn').std())
    .with_columns(dataset=pl.lit('')))

ANS['12.5.b'] = ANOVA(DATA['12.5 - aggregated'])
ANS['12.5.b']

dataset,source,SS,dof,MS,F
str,str,f64,u32,f64,f64
"""""","""treatment""",99.889305,2,49.944652,49.999257
"""""","""error""",37.9585,38,0.998908,null
"""""","""total""",137.847805,40,null,null


The p value is

In [30]:
stats.f(2, 38).sf(ANS['12.5.b'].item(0, 'F'))

2.2817841643408732e-11

So there are significant differences in hemoglobin levels.

#### (c)

In [31]:
chart_base = (
    alt.Chart(
        DATA['12.5']
        .with_columns(
            residual=col('hemoglbn') - col('hemoglbn').mean().over('Group'))
        .with_columns(
            normal_score=(col('residual').rank() / (pl.len() + 1)).map_batches(
                lambda x: stats.norm.ppf(x))))
    .mark_circle())

(
    chart_base.encode(
        alt.X('Group').axis(labelAngle=0).title(None),
        alt.Y('residual'))
    .properties(width=200)
    | chart_base.encode(
        alt.X('residual'), 
        alt.Y('normal_score'))
)

alt.HConcatChart(...)

From the charts: the variances are roughly consistent across different diseases. The residuals are mostly normally distributed, too.

### 6.

In [32]:
DATA['12.6'] = pl.read_json('Ex12-6.json').explode(pl.all())
DATA['12.6'].head()

Height,Part
f64,str
69.0,"""Tenor1"""
72.0,"""Tenor1"""
71.0,"""Tenor1"""
66.0,"""Tenor1"""
76.0,"""Tenor1"""


#### (a)

In [33]:
(
    alt.Chart(DATA['12.6'])
    .mark_boxplot()
    .encode(
        alt.Y('Part'),
        alt.X('Height').scale(zero=False))
)

alt.Chart(...)

No obvious differences in height among the 4 parts.

#### (b)

In [34]:
DATA['12.6 - aggregated'] = (
    DATA['12.6']
    .group_by(col('Part').alias('group'))
    .agg(
        n=pl.len(),
        μ=col('Height').mean(),
        s=col('Height').std())
    .with_columns(dataset=pl.lit('')))

ANS['12.6.b'] = ANOVA(DATA['12.6 - aggregated'])
ANS['12.6.b']

dataset,source,SS,dof,MS,F
str,str,f64,u32,f64,f64
"""""","""treatment""",81.114717,3,27.038239,3.946515
"""""","""error""",705.67033,103,6.851168,null
"""""","""total""",786.785047,106,null,null


The p value is

In [35]:
stats.f(2, 103).sf(ANS['12.6.b'].item(0, 'F'))

0.02231249549059697

p < α = 0.05, so statistically, there are significant differences in height among the 4 vocal parts at the 0.05 confidence level. Note that the effect is not significant at the 0.01 level.

#### (c)

In [36]:
chart_base = (
    alt.Chart(
        DATA['12.6']
        .with_columns(
            residual=col('Height') - col('Height').mean().over('Part'))
        .with_columns(
            normal_score=(col('residual').rank() / (pl.len() + 1)).map_batches(
                lambda x: stats.norm.ppf(x))))
    .mark_circle())

(
    chart_base.encode(
        alt.X('Part').axis(labelAngle=0).title(None),
        alt.Y('residual'))
    .properties(width=200)
    | chart_base.encode(
        alt.X('residual'), 
        alt.Y('normal_score'))
)

alt.HConcatChart(...)

From the charts: the variances are roughly consistent across different vocal parts. The residuals are fairly normally distributed, too.

### 7.

In [37]:
DATA['12.7'] = pl.read_json('Ex12-7.json').explode(pl.all())
DATA['12.7'].head()

WtPre,WtDif,Group
f64,f64,str
80.5,1.7,"""Behavior"""
84.9,0.7,"""Behavior"""
81.5,-0.1,"""Behavior"""
82.6,-0.7,"""Behavior"""
79.9,-3.5,"""Behavior"""


#### (a)

In [38]:
(
    alt.Chart(DATA['12.7'])
    .mark_boxplot()
    .encode(
        alt.Y('Group'),
        alt.X('WtPre').scale(zero=False).title('weight before treatment'))
)

alt.Chart(...)

No obvious differences in weights among the 3 groups.

#### (b)

In [39]:
DATA['12.7 - pre'] = (
    DATA['12.7']
    .group_by('Group')
    .agg(
        n=pl.len(),
        μ=col('WtPre').mean(),
        s=col('WtPre').std())
    .with_columns(dataset=pl.lit('')))

ANS['12.7.b'] = ANOVA(DATA['12.7 - pre'])
ANS['12.7.b']

dataset,source,SS,dof,MS,F
str,str,f64,u32,f64,f64
"""""","""treatment""",31.247451,2,15.623725,0.498801
"""""","""error""",1503.482353,48,31.322549,null
"""""","""total""",1534.729804,50,null,null


The p value is

In [40]:
stats.f(2, 48).sf(ANS['12.7.b'].item(0, 'F'))

0.6103708574101225

p > α = 0.05, so there are no significant weight differences among the 3 treatment groups.

#### (c)

In [41]:
(
    alt.Chart(DATA['12.7'])
    .mark_boxplot()
    .encode(
        alt.Y('Group'),
        alt.X('WtDif').scale(zero=True).title('weight gain'))
)

alt.Chart(...)

The family treatment group seems to have made the most weight gains among the 3 groups. The behavior group is slightly better than the control.

#### (d)

In [42]:
DATA['12.7 - dif'] = (
    DATA['12.7']
    .group_by('Group')
    .agg(
        n=pl.len(),
        μ=col('WtDif').mean(),
        s=col('WtDif').std())
    .with_columns(dataset=pl.lit('')))

ANS['12.7.d'] = ANOVA(DATA['12.7 - dif'])
ANS['12.7.d']

dataset,source,SS,dof,MS,F
str,str,f64,u32,f64,f64
"""""","""treatment""",479.302353,2,239.651176,3.846414
"""""","""error""",2990.644706,48,62.305098,null
"""""","""total""",3469.947059,50,null,null


The p value is

In [43]:
stats.f(2, 48).sf(ANS['12.7.d'].item(0, 'F'))

0.02822327034132448

Since p < α = 0.05, there are significant differences in weight gains among the 3 treatment groups. These differences will not be significant at the 0.01 confidence level, however.

## 12.2 Multiple Comparisons of Means

In [44]:
def pairwise_comparison(
    data: pl.DataFrame,
    α: float=0.05,
    method: str|list[str]='tukey'
) -> pl.DataFrame:
    """ 
    returns pairwise comparison results of data as a DataFrame.

    input columns:
    - dataset: string id of dataset
    - group: distinct for different treatment groups
    - n: group size
    - μ: group mean
    - s: group standard deviation

    output columns:
    - dataset
    - pair: the pair of groups to be compared
    - μ_diff: difference between group means
    - followed by CIs for each method requested
    """
    def ci(μ, crit, se):
        return pl.concat_list(μ - crit*se, μ + crit*se)

    # critical values calculation by comparison methods
    crit = {
        'lsd': col('dof').map_batches(lambda dof: stats.t.ppf(1-α/2, df=dof)),
        'bonferroni': (
            pl.struct(col('dof'), col('pairs'))
            .map_batches(lambda x:
                stats.t.ppf(
                    q=1-α/2/x.struct.field('pairs'), 
                    df=x.struct.field('dof')))),
        'tukey': (
            pl.struct(col('dof'), col('n_groups'))
            .map_batches(lambda x: 
                stats.studentized_range.ppf(
                    q=1-α, 
                    k=x.struct.field('n_groups'), 
                    df=x.struct.field('dof'))) 
            / np.sqrt(2))}

    if isinstance(method, str):
        method = [method]

    return (
        data
        .with_columns(
            n_total=col('n').sum().over('dataset'),
            n_groups=pl.len().over('dataset'),
            s_pool=((col('s')**2 * (col('n')-1)).sum() / (col('n') - 1).sum()).sqrt().over('dataset'))
        .join(data, on='dataset')
        .filter(col('group') < col('group_right'))
        .with_columns(
            μ_diff = col('μ') - col('μ_right'),
            dof = col('n_total') - col('n_groups'),
            pairs = col('n_groups') * (col('n_groups') - 1) / 2,
            se = col('s_pool') * (1 / col('n') + 1 / col('n_right')).sqrt())
        .select(
            'dataset',
            pl.concat_list(col('group'), col('group_right')).alias('pair'),
            'μ_diff',
            *[ci(col('μ_diff'), crit.get(m), col('se')).alias(m) for m in set(method)]))

In [45]:
def stepwise_comparison(
    data: pl.DataFrame, 
    α: float=0.05, 
    method: str='ryan'
) -> pl.DataFrame:
    """
    returns stepwise comparison results of data as a DataFrame.

    input columns:
    - dataset: name of dataset
    - group: distinct for different treatment groups
    - n: group size
    - μ: group mean
    - s: group standard deviation

    output columns:
    - dataset: name of dataset
    - pair: a pair of groups [g1, g2] that are significantly different
    """
            
    def significant_pairs(data: pl.Series) -> pl.Series:
        """
        input:
        data.group
        data.μ
        """
        data = data.struct.unnest().sort('μ')

        n_total, n_groups, s_pool= (
            data.select(
                col('n').sum().alias('n_total'),
                pl.len().alias('n_groups'),
                ((col('s')**2 * (col('n')-1)).sum() / (col('n') - 1).sum()).sqrt()
                .alias('s_pool'))
            .row(0))

        stepwise_method = {
            'duncan': lambda p: 1 - (1 - α)**(p - 1),
            'ryan': lambda p: α if p >= n_groups - 1 else p * α / n_groups}

        def stepwise(start: int, end: int) -> list[list[str]]:
            if start >= end - 1:
                return []

            g1, n1, μ1, _ = data.row(start)
            g2, n2, μ2, _ = data.row(end-1)
                            
            if μ2 - μ1 > (
                s_pool 
                * np.sqrt(0.5/n1 + 0.5/n2) 
                * stats.studentized_range.ppf(
                    q = 1 - stepwise_method[method](end-start),
                    k = n_groups,
                    df = n_total - n_groups)):
                return (
                    [[g1, g2] if g1 < g2 else [g2, g1]] 
                    + stepwise(start+1, end) 
                    + stepwise(start, end-1))
            else:
                return []

        return pl.Series('pair', stepwise(0, n_groups))

    return (
        data.with_columns(
            pl.struct('group', 'n', 'μ', 's')
            .alias('structure'))
        .group_by('dataset')
        .agg(
            col('structure')
            .map_elements(
                lambda data: significant_pairs(data))
            .alias('pair'))
        .explode('pair'))

## 12.2 Solutions to Exercises

### 8.

In [46]:
DATA['12.1 - long']

group,n,dataset,μ,s
str,i64,str,f64,f64
"""Shelf1""",20,"""fiber""",1.68,1.166
"""Shelf2""",20,"""fiber""",0.95,1.162
"""Shelf3""",20,"""fiber""",2.17,1.277
"""Shelf1""",20,"""sugar""",4.8,2.138
"""Shelf2""",20,"""sugar""",9.85,1.985
"""Shelf3""",20,"""sugar""",6.1,1.865


In [47]:
pairwise_comparison(DATA['12.1 - long'], method=['bonferroni', 'tukey'])

dataset,pair,μ_diff,tukey,bonferroni
str,list[str],f64,list[f64],list[f64]
"""fiber""","[""Shelf1"", ""Shelf2""]",0.73,"[-0.185339, 1.645339]","[-0.208263, 1.668263]"
"""fiber""","[""Shelf1"", ""Shelf3""]",-0.49,"[-1.405339, 0.425339]","[-1.428263, 0.448263]"
"""fiber""","[""Shelf2"", ""Shelf3""]",-1.22,"[-2.135339, -0.304661]","[-2.158263, -0.281737]"
"""sugar""","[""Shelf1"", ""Shelf2""]",-5.05,"[-6.571286, -3.528714]","[-6.609387, -3.490613]"
"""sugar""","[""Shelf1"", ""Shelf3""]",-1.3,"[-2.821286, 0.221286]","[-2.859387, 0.259387]"
"""sugar""","[""Shelf2"", ""Shelf3""]",3.75,"[2.228714, 5.271286]","[2.190613, 5.309387]"


A comparison is significant if the CI does not include 0.
- For fiber content:
    - Bonferroni method: shelves 2 and 3 are different.
    - Tukey method: shelves 2 and 3 are different.
- For sugar content:
    - Bonferroni method: shelf 2 is different from shelves 1 and 3.
    - Tukey method: shelf 2 is different from shelves 1 and 3.

### 9.

In [48]:
DATA['12.4 - aggregated']

group,n,μ,s,dataset
str,u32,f64,f64,str
"""II""",8,40.10375,0.531385,""""""
"""III""",10,38.892,0.510812,""""""
"""I""",12,37.313333,0.572797,""""""


In [49]:
pairwise_comparison(DATA['12.4 - aggregated'], method='tukey')

dataset,pair,μ_diff,tukey
str,list[str],f64,list[f64]
"""""","[""I"", ""II""]",-2.790417,"[-3.403887, -2.176946]"
"""""","[""II"", ""III""]",1.21175,"[0.574213, 1.849287]"
"""""","[""I"", ""III""]",-1.578667,"[-2.154153, -1.00318]"


Tukey method: all sites are different from each other.

In [50]:
stepwise_comparison(DATA['12.4 - aggregated'], method='ryan')

dataset,pair
str,list[str]
"""""","[""I"", ""II""]"
"""""","[""II"", ""III""]"
"""""","[""I"", ""III""]"


Ryan method: same conclusion -- all sites are different.

### 10.

In [51]:
DATA['12.5 - aggregated']

group,n,μ,s,dataset
str,u32,f64,f64,str
"""HBSC""",15,12.3,0.941883,""""""
"""HBS""",10,10.63,1.284134,""""""
"""HBSS""",16,8.7125,0.844492,""""""


In [52]:
pairwise_comparison(DATA['12.5 - aggregated'], method=['bonferroni', 'tukey', 'lsd'], α=0.01)

dataset,pair,μ_diff,tukey,lsd,bonferroni
str,list[str],f64,list[f64],list[f64],list[f64]
"""""","[""HBS"", ""HBSC""]",-1.67,"[-2.933945, -0.406055]","[-2.776384, -0.563616]","[-2.948023, -0.391977]"
"""""","[""HBSC"", ""HBSS""]",3.5875,"[2.474798, 4.700202]","[2.613505, 4.561495]","[2.462404, 4.712596]"
"""""","[""HBS"", ""HBSS""]",1.9175,"[0.669455, 3.165545]","[0.825033, 3.009967]","[0.655553, 3.179447]"


All three methods (Tukey, Bonferroni, and LSD) reach the same conclusion -- all pairwise comparisons are different (CI does not include 0). But Tukey and Bonferroni give similar CIs with Bonferroni slightly wider. The LSD gives much narrower CIs (most sensitive).

### 11.

In [53]:
DATA['12.6 - aggregated']

group,n,μ,s,dataset
str,u32,f64,f64,str
"""Tenor2""",21,69.904762,2.071346,""""""
"""Bass1""",39,70.717949,2.361408,""""""
"""Tenor1""",21,68.904762,3.330237,""""""
"""Bass2""",26,71.384615,2.728764,""""""


In [54]:
pairwise_comparison(DATA['12.6 - aggregated'], α=0.1, method='tukey')

dataset,pair,μ_diff,tukey
str,list[str],f64,list[f64]
"""""","[""Bass1"", ""Tenor2""]",0.813187,"[-0.83086, 2.457233]"
"""""","[""Tenor1"", ""Tenor2""]",-1.0,"[-2.874502, 0.874502]"
"""""","[""Bass2"", ""Tenor2""]",1.479853,"[-0.302251, 3.261958]"
"""""","[""Bass1"", ""Tenor1""]",1.813187,"[0.16914, 3.457233]"
"""""","[""Bass2"", ""Tenor1""]",2.479853,"[0.697749, 4.261958]"
"""""","[""Bass1"", ""Bass2""]",-0.666667,"[-2.204531, 0.871198]"
